## BMEB W4020: Circuits in the Brain 2020 Fall Homework #1

Author: [Shashwat Shukla](shashwat.shukla@columbia.edu), [Bruce Yi Bu](yb2520@columbia.edu), [Pranav Deevi](pid2104@columbia.edu)

Based on Previous Work by: Yinghao Li, Tingkai Liu, Mehmet Kerem Turkcan, Chung-Heng Yeh, Konstantinos Psychas

Copyright 2023 Shashwat Shukla, Tingkai Liu, Bruce Yi Bu, Pranav Deevi and Aurel A. Lazar

### Names and UNIs

- Lawrence Xu, lzx2002

(Please include the names and UNIs of all team members here; **double click** on **markdown** blocks to edit)

### Before you start...
#### The code skeleton 
* we provide a skeleton of the code for HW1 and all future assignments (excluding the projects)
* you will find helper functions as well as headers of functions that need to be completed (look for "**TO DO**" comments)
* note that you are not required to follow this sequence; the skeleton is just provided for convenience but we highly recommend using it, and you are free to modify it or add custom functions to it.

#### Miscellaneous (attention)
* please remember to include your name in the corresponding field at the start of this notebook, otherwize an automatic 0 grade will be received for all team members
* please post your questions on Ed Discussions, and attend the office hours (every Thursday evening)
* please clearly present your solutions and answers with proper use of plots, markdown and comments
* please start the assignment as soon as possible, submission deadlines are strictly followed and late submissions will receive a 10% penalty for each 24 hours post the deadline
* please email the TA/CAs at least 48hrs ahead of the deadline if you cannot submit an assignment on time (due to exam schedules or overwhelming deadlines, etc), we are flexible as long as we are informed.

## PROBLEM: The Reduced Hodgkin-Huxley Model (Wilson Neuron) 
### Part 1: Solving the Reduced Hodgkin-Huxley Model Numerically
In this problem you are asked to implement (solve) a reduced Hodgkin-Huxley neuron model (**Wilson neuron**) that is a further simplification of the Rinzel model. The Wilson neuron model is described by the differential equations:

\begin{equation}
\begin{split}
C\frac{dV}{dt} &= -g_{Na}\cdot(17.81+47.71V+32.63V^2)\cdot(V-E_{Na}) - g_R\cdot R\cdot(V-E_R) + I(t) \\
\tau_R\frac{dR}{dt} &= -R + 1.35V + 1.03 \\
\end{split}
\end{equation}
where 


    
|$g_{Na}$|$E_{Na}$|$g_R$|$E_R$|$\tau_R$|$C$|
|:----|:-----|:---|:---|:-----|:-----|
|<img style="width:100px">1 $mS/cm^2$ |<img style="width:100px">0.55 $mV$|<img style="width:100px">26 $mS/cm^2 $|<img style="width:100px">-0.92 $mV$|<img style="width:100px">1.9 $ms$|<img style="width:100px">0.8 $\mu F/cm^2$|
    
Create an input current of the form
$$I(t)= \sum^{10}_{k=0}a_k \frac{\sin \Omega (t-kT)}{\Omega (t-kT)}$$
where $a_k = constant (pA), 0 \leq k \leq 10$, $\Omega = 2 \pi \cdot 60$ Hz, $T = \pi/\Omega$,
$t \in [0, 200]$ ms. 

1. Implement the Wilson neuron model defined by the equations above. You are asked to implement [Euler's method](https://en.wikipedia.org/wiki/Euler_method) for solving differential equations numerically. Essentially, for the differential equation $\frac{dx}{dt} = f(x) + I(t)$, we iteratively evaluate the value of $x(t+\Delta t)$ given the previous value $x(t)$ and the gradient of $f(x(t)) + I(t)$ as $\frac{x(t+\Delta t) - x(t)}{\Delta t} = f(x(t)) + I(t)$.

2. Assuming that $a_k=constant, -M \le m \le M$, with initial conditions $V(0)=-0.7mV, R(0)=0.088 (unitless)$, plot the input and the output of the Wilson neuron on the time interval [0, 200] ms, using an $a_k$ such that the neuron robustly spikes. 
    * Make sure to show one plot featuring both input current and membrane voltage. You can use the `.twinx()` function in `matplotlib` (google documentation if you wish) to create two y-axes in one plot.
    * We recommend experimenting with a wide range of $a_k$ values to see its effect on the voltage spikes.
    * Since Wilson model's voltage scale is *100 smaller* than the biophysically accurate range (e.g., $V(0)=-70mV$ would be the biophysical norm), feel free to scale up in your plot the voltage output by 100 times.  
3. Empirically determine a range of values for the $a_k$'s that makes the neuron fire. Plot the total number of spikes fired in the interval [0, 200] ms for a broad range of $a_k = constant$ values. For the same range of input current values, show the following
    1. spike count
    2. spike rate (spikes per second)
    3. spike amplitude (Voltage value at spike time)
4. Assume the input current is $I(t) = c\cdot u(t)$, where $u(t)$ is a [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function), and $c$ is amplitude of the injected current in $pA$. Find a range of values for $c$ such that the Wilson neuron exhibits periodic spiking (limit cycle). Draw the limit cycle in the $V$ vs. $R$ plane for at least 3 different values of $c$, comment on the effect of the amplitude $c$ on the limit cycle and the membrane voltage based on your results.

In [12]:
# Import libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

np.random.seed(0)  # fix random seed

In [6]:
# Define spike detection function
def spike_detect(v, thresh):
    x = (v[2:-1] > v[1:-2]) * (v[2:-1] > v[3:]) * (v[2:-1] > thresh)
    x = np.append(False, x)
    x = np.append(x, False)
    return x


In [39]:
# TODO - define a funciton to generate the input current
# the functions should take the following forms

#def I_in(t,a,K,omega):
#   input: (t,a,K,omega), a is a vector of all ak values
#   output: I

def I_in(t, a, k, omega):
    I = 0.0
    for j in range(k):
        I = I + a[j]*(np.sin(omega) * (t - j * (np.pi / omega))) / (omega * (t - j * (np.pi / omega)))
    return I



In [27]:
g_Na = 1 #mS/cm^2
E_Na = 0.55 #mV
g_R = 26 #mS/cm^2
E_R = -0.92 #mV
th_R = 1.9 #ms
C = 0.8 #muF / cm^2
k = 10

omega = 2 * np.pi / (1000 / 60)

#dV/dt = f(t, V)
#dR/dt = f(t, R)

In [16]:
# Wilson model dynamics
# Create a function to define the wilson model 
# that takes in injection current and time vector as inpuit
# and return V and R as vectors 
# Pay attention to time scale (ms vs s)!

t = np.linspace(0, 1, 200)

V0 = -0.7 #mV
R0 = 0.088

def dVdt(V, t, I_ext, g_Na, E_Na, g_R, E_R, th_R, C, R):
    dVdt = (-g_Na * (17.81 + (47.71 * V) + (32.63 * (V ** 2))) * (V - E_Na) - (g_R * R * (V - E_R)) + I_ext) / C
    
    return dVdt

def dRdt(V, t, I_ext, g_Na, E_Na, g_R, E_R, th_R, C, R):
    dRdt = (-R + 1.35 * V + 1.03) / th_R

 
    
def wilson(t, I_ext):
    V = odeint(dVdt, )

    
    
                                                                               
    return V, R

In [21]:
def dWdt(W, t):
    
    g_Na = 1 #mS/cm^2
    E_Na = 0.55 #mV
    g_R = 26 #mS/cm^2
    E_R = -0.92 #mV
    th_R = 1.9 #ms
    C = 0.8 #muF / cm^2

    V, R = W
    dVdt = (-g_Na * (17.81 + (47.71 * V) + (32.63 * (V ** 2))) * (V - E_Na) - (g_R * R * (V - E_R)) + I_ext) / C
    dRdt = (-R + 1.35 * V + 1.03) / th_R

    return dVdt, dRdt


In [38]:
V_0 = -0.7
R_0 = 0.088
W_0 = (V_0, R_0)

a = 1
I_ext = I_in(t, a, k, omega)
t = np.linspace(0, 1, 200)
sol = odeint(dWdt, y0=W_0, t=t)

C:\Users\lawre\AppData\Local\Temp\ipykernel_6124\3950450935.py:11: RuntimeWarning: invalid value encountered in divide
  I = I + a*(np.sin(omega) * (t - j * (np.pi / omega))) / (omega * (t - j * (np.pi / omega)))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [36]:
print(type()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3811138061.py, line 1)

In [5]:
# TO DO: simulate the model and detect the spikes


In [6]:
# TO DO: plot the results

In [7]:
# Determine the range where the neuron robustly fires
# TO DO: perform the experiment by first defining the range of amplitudes

In [8]:
# TO DO: plot the results

In [9]:
# For Heaviside step function current input, determine range of c to make the model periodically spike
# We recomment picking three values to show where the neuron
# 1. c smaller than the range
# 2. c within the range
# 3. c over the range
# TO DO: for each case above, plot limit cycle (V vs R) and the membrane voltage vs time

#### TO DO: (double click to edit)
Comment (in markdown) on the effect of c on the limit cycle and membrane voltage that you observe. 

### Part 2: Analyzing the Internal State of the Reduced Hodgkin-Huxley Model

As done in Chapter 3 for the Na and K channels of a Hodgkin-Huxley neuron, in this part, we model the variable `R` of the Reduced Hodgkin-Huxley neuron as a memconductance. 
1. Write down Ohm's law for $I_{R}$ and the differential equation for $I_R$, treating the current $I_R$ as input. 
2. Assuming (a) ramp and (b) spike train input currents, plot the voltage $V_R=V-E_R$ versus current $I_R$. Is $V_R$ a function of the current $I_R$?
3. Is the flux $\phi_R (t) = \int_0^t V_R(s) ds$ a function of the charge $q_R (t) = \int_0^t I_R (s) ds$?

#### Q1: (double click to edit)
TO DO: Your answer here.
(You can look up markdown syntax for equations online or simply follow the previous question prompts)

In [1]:
# Q2: Ramp current input
#TO DO: 

In [2]:
# Q2: Sike train current input
# TO DO: generate spike train with poison process

#### Q2: (double click to edit)
TO DO: observation on $V_R$ and $I_R$

#### Q3: (double click to edit)
TO DO: your answer here